<a href="https://colab.research.google.com/github/mkhan5412/Deep-learning/blob/master/Text%20Classification%20using%20GloVe%20%2B%20CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **EXTRACTION**

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [5]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [6]:
os.listdir("/content/gdrive/My Drive/Kaggle")

['glove.twitter.27B.25d.txt',
 'glove.twitter.27B.200d.txt',
 'kaggle.json',
 '.ipynb_checkpoints',
 'bbc-fulltext (document classification)',
 'bbc']

In [ ]:
!kaggle datasets download -d shivamkushwaha/bbc-full-text-document-classification

  0% 0.00/5.59M [00:00<?, ?B/s]
100% 5.59M/5.59M [00:00<00:00, 51.5MB/s]


In [ ]:
!unzip \*.zip  && rm *.zip

unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.


In [7]:
import numpy as np 
import pandas as pd 
import re
import nltk
from nltk.corpus import stopwords
import seaborn as sns
import matplotlib.pyplot as pl
import os

In [8]:
for dirname, _, filenames in os.walk('/content/gdrive/MyDrive/Kaggle/bbc'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/gdrive/MyDrive/Kaggle/bbc/README.TXT
/content/gdrive/MyDrive/Kaggle/bbc/business/007.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/008.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/009.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/012.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/010.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/011.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/014.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/013.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/015.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/016.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/018.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/019.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/017.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/022.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/020.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/021.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/024.txt
/content/gdrive/MyDrive/Kaggle/bbc/business/023.txt
/content/gdrive/My

In [9]:
path = '/content/gdrive/MyDrive/Kaggle/bbc'
os.listdir(path)

['business', 'entertainment', 'politics', 'sport', 'tech', 'README.TXT']

In [22]:
from collections import defaultdict
dicts=defaultdict(list)

for dir_name,_,file_names in os.walk(path):
  for file in file_names:
    dicts['categories'].append(os.path.basename(dir_name))
    name=os.path.splitext(file)[0]
    dicts['doc_id'].append(name)
    path=os.path.join(dir_name,file)
        
    with open(path,'r', encoding='latin-1') as file:
      dicts['text'].append(file.read())

In [24]:
df=pd.DataFrame.from_dict(dicts)
df.head()

""


In [ ]:
df.drop(0, inplace=True)
df.head()

,categories,doc_id,text
1,business,007,Jobs growth still slow in the US\n\nThe US cre...
2,business,008,"India calls for fair trade rules\n\nIndia, whi..."
3,business,009,Ethiopia's crop production up 24%\n\nEthiopia ...
4,business,012,Indonesians face fuel price rise\n\nIndonesia'...
5,business,010,Court rejects $280bn tobacco case\n\nA US gove...


In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

,categories,doc_id,text
0,business,007,Jobs growth still slow in the US\n\nThe US cre...
1,business,008,"India calls for fair trade rules\n\nIndia, whi..."
2,business,009,Ethiopia's crop production up 24%\n\nEthiopia ...
3,business,012,Indonesians face fuel price rise\n\nIndonesia'...
4,business,010,Court rejects $280bn tobacco case\n\nA US gove...


In [ ]:
len(df)

2225

In [ ]:
df.drop('doc_id', axis=1, inplace=True)
df.head()

,categories,text
0,business,Jobs growth still slow in the US\n\nThe US cre...
1,business,"India calls for fair trade rules\n\nIndia, whi..."
2,business,Ethiopia's crop production up 24%\n\nEthiopia ...
3,business,Indonesians face fuel price rise\n\nIndonesia'...
4,business,Court rejects $280bn tobacco case\n\nA US gove...


In [ ]:
def returnTitle(data):
    data=data.split('\n')[0]
    return data
def returnArticle(data):
    data=' '.join(data.split('\n')[1:])
    return data

    
df['title']=df['text'].apply(lambda x:returnTitle(x))
df['article']=df['text'].apply(lambda x: returnArticle(x))

In [ ]:
df.head()

,categories,text,title,article
0,business,Jobs growth still slow in the US\n\nThe US cre...,Jobs growth still slow in the US,The US created fewer jobs than expected in Ja...
1,business,"India calls for fair trade rules\n\nIndia, whi...",India calls for fair trade rules,"India, which attends the G7 meeting of seven ..."
2,business,Ethiopia's crop production up 24%\n\nEthiopia ...,Ethiopia's crop production up 24%,Ethiopia produced 14.27 million tonnes of cro...
3,business,Indonesians face fuel price rise\n\nIndonesia'...,Indonesians face fuel price rise,Indonesia's government has confirmed it is co...
4,business,Court rejects $280bn tobacco case\n\nA US gove...,Court rejects $280bn tobacco case,A US government claim accusing the country's ...


### **CLEANING**

In [ ]:
df.drop('text', axis=1, inplace=True)
df.head()

,categories,title,article
0,business,Jobs growth still slow in the US,The US created fewer jobs than expected in Ja...
1,business,India calls for fair trade rules,"India, which attends the G7 meeting of seven ..."
2,business,Ethiopia's crop production up 24%,Ethiopia produced 14.27 million tonnes of cro...
3,business,Indonesians face fuel price rise,Indonesia's government has confirmed it is co...
4,business,Court rejects $280bn tobacco case,A US government claim accusing the country's ...


In [ ]:
print(pd.Series({c: df[c].map(lambda x: len(str(x))).max() for c in df}).sort_values(ascending =False))

article       25454
title            52
categories       13
dtype: int64


In [ ]:
df['categories'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: categories, dtype: int64

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [ ]:
df1 = df.copy()

In [ ]:
import re
import string

stopword = stopwords.words('english')

def clean_text(text):
    # remove line breaks \r \n \t remove from string 
    text = text.replace('\\r', ' ')
    text = text.replace('\\"', ' ')
    text = text.replace('\\t', ' ')
    text = text.replace('\\n', ' ')

    # remove stopwords
    text = ' '.join(word for word in text.split() if word not in set(stopword))

    # remove special chars
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    text = text.lower()

    return text

In [ ]:
df1['title']

0        Jobs growth still slow in the US
1        India calls for fair trade rules
2       Ethiopia's crop production up 24%
3        Indonesians face fuel price rise
4       Court rejects $280bn tobacco case
                      ...                
2220     BT program to beat dialler scams
2221      Spam e-mails tempt net shoppers
2222     Losing yourself in online gaming
2223              Be careful how you code
2224      US cyber security chief resigns
Name: title, Length: 2225, dtype: object

In [ ]:
from tqdm import tqdm

processed_titles = []
for title in (df['article'].values):
    processed_title = clean_text(title)
    processed_titles.append(processed_title)

In [ ]:
processed_titles[0]

'the us created fewer jobs expected january fall jobseekers pushed unemployment rate lowest level three years according labor department figures us firms added 146 000 jobs january the gain non farm payrolls market expectations 190 000 new jobs nevertheless enough push unemployment rate 5 2 lowest level since september 2001 the job gains mean president bush celebrate albeit fine margin net growth jobs us economy first term office he presided net fall jobs last november presidential election first president since herbert hoover as result job creation became key issue last year election however adding december january figures administration first term jobs record ended positive territory the labor department also said revised jobs gains december 2004 157 000 133 000 analysts said growth new jobs strong could expected given favourable economic conditions it suggests employment continuing expand moderate pace said rick egelton deputy chief economist bmo financial group we getting boost emp

In [ ]:
df1['article_new'] = processed_titles
df1.head()

,categories,title,article,article_new
0,business,Jobs growth still slow in the US,The US created fewer jobs than expected in Ja...,the us created fewer jobs expected january fal...
1,business,India calls for fair trade rules,"India, which attends the G7 meeting of seven ...",india attends g7 meeting seven leading industr...
2,business,Ethiopia's crop production up 24%,Ethiopia produced 14.27 million tonnes of cro...,ethiopia produced 14 27 million tonnes crops 2...
3,business,Indonesians face fuel price rise,Indonesia's government has confirmed it is co...,indonesia government confirmed considering rai...
4,business,Court rejects $280bn tobacco case,A US government claim accusing the country's ...,a us government claim accusing country biggest...


In [ ]:
df1.head()

,categories,title,article,article_new
0,business,Jobs growth still slow in the US,The US created fewer jobs than expected in Ja...,the us created fewer jobs expected january fal...
1,business,India calls for fair trade rules,"India, which attends the G7 meeting of seven ...",india attends g7 meeting seven leading industr...
2,business,Ethiopia's crop production up 24%,Ethiopia produced 14.27 million tonnes of cro...,ethiopia produced 14 27 million tonnes crops 2...
3,business,Indonesians face fuel price rise,Indonesia's government has confirmed it is co...,indonesia government confirmed considering rai...
4,business,Court rejects $280bn tobacco case,A US government claim accusing the country's ...,a us government claim accusing country biggest...


In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
df1['categories'] = enc.fit_transform(df1['categories'])

df1['categories'].head()

,categories,title,article,article_new
0,0,Jobs growth still slow in the US,The US created fewer jobs than expected in Ja...,the us created fewer jobs expected january fal...
1,0,India calls for fair trade rules,"India, which attends the G7 meeting of seven ...",india attends g7 meeting seven leading industr...
2,0,Ethiopia's crop production up 24%,Ethiopia produced 14.27 million tonnes of cro...,ethiopia produced 14 27 million tonnes crops 2...
3,0,Indonesians face fuel price rise,Indonesia's government has confirmed it is co...,indonesia government confirmed considering rai...
4,0,Court rejects $280bn tobacco case,A US government claim accusing the country's ...,a us government claim accusing country biggest...


In [ ]:
text = df1['article_new'].tolist()
text[0]

'the us created fewer jobs expected january fall jobseekers pushed unemployment rate lowest level three years according labor department figures us firms added 146 000 jobs january the gain non farm payrolls market expectations 190 000 new jobs nevertheless enough push unemployment rate 5 2 lowest level since september 2001 the job gains mean president bush celebrate albeit fine margin net growth jobs us economy first term office he presided net fall jobs last november presidential election first president since herbert hoover as result job creation became key issue last year election however adding december january figures administration first term jobs record ended positive territory the labor department also said revised jobs gains december 2004 157 000 133 000 analysts said growth new jobs strong could expected given favourable economic conditions it suggests employment continuing expand moderate pace said rick egelton deputy chief economist bmo financial group we getting boost emp

In [ ]:
from keras.preprocessing.text import Tokenizer

token  = Tokenizer()
token.fit_on_texts(text)

In [ ]:
voc = len(token.word_index) + 1
voc

29334

In [ ]:
encoded_text = token.texts_to_sequences(text)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

max_len = 17257
X = pad_sequences(encoded_text, padding='pre', maxlen = 500)
Y = df1.categories

In [ ]:
%%time 
file = open("/content/gdrive/MyDrive/Kaggle/glove.twitter.27B.25d.txt", encoding='utf-8')
glove_vectors = dict()

for line in file:
  values = line.split()
  word = values[0]
  vectors = np.array(values[1:])
  glove_vectors[word] = vectors
file.close()

CPU times: user 7.84 s, sys: 254 ms, total: 8.1 s
Wall time: 8.11 s


In [ ]:
word_vector_matrix = np.zeros((voc, 25))

for word, index in token.word_index.items():
  vector = glove_vectors.get(word)
  if vector is not None:
    word_vector_matrix[index] = vectors  

###**MODELLING**

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=123)

Y_train = to_categorical(Y_train)
Y_test  = to_categorical(Y_test)

print('Shape of X_train is = ',X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

NameError: ignored

In [ ]:
from keras.layers import Dense, LSTM, Dropout, SimpleRNN, Activation, Reshape, Conv1D, MaxPooling1D, Flatten, Embedding
from keras.models import Sequential

vector_size = 200

classifier = Sequential()

classifier.add(Embedding(voc,25, input_length=500))
classifier.add(Dropout(0.2))

classifier.add(Conv1D(64,5, padding='valid'))
classifier.add(MaxPooling1D())

classifier.add(Flatten())

classifier.add(Dropout(0.2))

classifier.add(Dense(5))
classifier.add(Activation('softmax'))

classifier.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

classifier.summary()

In [ ]:
classifier.fit(X_train,Y_train, validation_data=(X_test, Y_test), epochs=20, batch_size=10, verbose=2)

Epoch 1/20
178/178 - 2s - loss: 1.1842 - accuracy: 0.5112 - val_loss: 0.3937 - val_accuracy: 0.8719
Epoch 2/20
178/178 - 1s - loss: 0.0912 - accuracy: 0.9781 - val_loss: 0.1277 - val_accuracy: 0.9528
Epoch 3/20
178/178 - 1s - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.0946 - val_accuracy: 0.9708
Epoch 4/20
178/178 - 1s - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.0918 - val_accuracy: 0.9685
Epoch 5/20
178/178 - 1s - loss: 6.4987e-04 - accuracy: 1.0000 - val_loss: 0.0905 - val_accuracy: 0.9685
Epoch 6/20
178/178 - 1s - loss: 4.2344e-04 - accuracy: 1.0000 - val_loss: 0.0896 - val_accuracy: 0.9685
Epoch 7/20
178/178 - 1s - loss: 3.1698e-04 - accuracy: 1.0000 - val_loss: 0.0890 - val_accuracy: 0.9685
Epoch 8/20
178/178 - 1s - loss: 2.3311e-04 - accuracy: 1.0000 - val_loss: 0.0888 - val_accuracy: 0.9730
Epoch 9/20
178/178 - 1s - loss: 1.9059e-04 - accuracy: 1.0000 - val_loss: 0.0890 - val_accuracy: 0.9685
Epoch 10/20
178/178 - 1s - loss: 1.4655e-04 - accuracy: 1.0000 - val_loss: 0.088

In [ ]:
df1.head()

,categories,title,article,article_new
0,business,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...,quarterly profits us media giant timewarner ju...
1,business,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...,the dollar hit highest level euro almost three...
2,business,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...,the owners embattled russian oil giant yukos a...
3,business,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...,british airways blamed high fuel prices 40 dro...
4,business,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,shares uk drinks food firm allied domecq risen...


In [ ]:
df1['categories'].value_counts()

3    511
0    510
2    417
4    401
1    386
Name: categories, dtype: int64

In [ ]:
print(token.index_word)

{1: 'the', 2: 'said', 3: 'i', 4: 'mr', 5: 'would', 6: 'it', 7: 'year', 8: 'also', 9: 'people', 10: 'new', 11: 'one', 12: 'us', 13: 'but', 14: 'he', 15: 'could', 16: 'last', 17: 'first', 18: 'time', 19: 'two', 20: 'in', 21: 'a', 22: 'world', 23: 'government', 24: 'we', 25: 'uk', 26: 'years', 27: 'best', 28: 'make', 29: 'told', 30: '1', 31: 'like', 32: 'get', 33: 'made', 34: 'game', 35: 'film', 36: 'many', 37: 'three', 38: 'back', 39: 'music', 40: '000', 41: 'next', 42: 'labour', 43: 'well', 44: 'number', 45: 'bbc', 46: '2', 47: 'way', 48: 'take', 49: 'added', 50: 'set', 51: 'this', 52: 'and', 53: 'market', 54: 'company', 55: 'may', 56: 'there', 57: 'says', 58: 'home', 59: 'good', 60: '6', 61: 'election', 62: 'going', 63: '2004', 64: 'party', 65: 'much', 66: 'still', 67: 'games', 68: 'since', 69: 'second', 70: 'work', 71: 'that', 72: 'england', 73: 'go', 74: 'think', 75: 'week', 76: 'win', 77: 'firm', 78: 'use', 79: '3', 80: 'say', 81: 'million', 82: 'they', 83: 'part', 84: 'show', 85: '

In [ ]:
print(encoded_text[:2])

[[1, 12, 838, 2734, 438, 107, 235, 642, 14782, 1915, 1369, 373, 2350, 463, 37, 26, 142, 6045, 791, 219, 12, 164, 49, 12402, 40, 438, 235, 1, 1829, 867, 4941, 10756, 53, 1505, 8001, 40, 10, 438, 4733, 397, 1731, 1369, 373, 97, 46, 2350, 463, 68, 577, 508, 1, 427, 2940, 599, 203, 818, 4734, 5713, 1830, 4381, 140, 129, 438, 12, 139, 17, 434, 210, 14, 14783, 140, 642, 438, 16, 440, 3486, 61, 17, 203, 68, 9582, 14784, 211, 436, 427, 2351, 811, 364, 328, 16, 7, 61, 104, 1230, 245, 235, 219, 1703, 17, 434, 438, 150, 1294, 917, 4382, 1, 6045, 791, 8, 2, 3487, 438, 2940, 245, 63, 10757, 40, 8685, 40, 278, 2, 129, 10, 438, 307, 15, 107, 228, 6874, 168, 1481, 6, 1130, 1916, 1423, 3590, 5714, 1589, 2, 4942, 14785, 1545, 127, 1443, 14786, 318, 110, 24, 401, 717, 1916, 5, 171, 228, 404, 707, 382, 66, 2294, 404, 286, 373, 1444, 1, 139, 2154, 5714, 8002, 829, 427, 129, 2, 3027, 12403, 203, 12404, 2799, 71, 418, 1314, 44, 10, 1606, 781], [501, 10758, 3372, 369, 698, 667, 5423, 258, 473, 1295, 19295, 47

In [ ]:
X.shape

(2225, 500)

In [ ]:
glove_vectors = dict()

In [ ]:
len(glove_vectors.keys())

1193514

In [ ]:
glove_vectors.get('you').shape

(25,)

In [ ]:
import gensim

In [ ]:
model = gensim.models.Word2Vec(df['article'], min_count=1, size=32)

In [ ]:
gensim.models.Word2Vec.load(df['article'])

AttributeError: ignored

### **TRIAL & ERROR**

In [ ]:
df2 = df.copy()

In [ ]:
voc = df2['article'].str.count(' ').sum()
voc

853872

In [ ]:
from tqdm import tqdm

processed_titles = []
for title in (df2['article'].values):
    processed_title = clean_text(title)
    processed_titles.append(processed_title)

In [ ]:
df2['article'] = processed_titles

In [ ]:
df2

,categories,title,article
0,business,Ad sales boost Time Warner profit,quarterly profits us media giant timewarner ju...
1,business,Dollar gains on Greenspan speech,the dollar hit highest level euro almost three...
2,business,Yukos unit buyer faces loan claim,the owners embattled russian oil giant yukos a...
3,business,Pernod takeover talk lifts Domecq,shares uk drinks food firm allied domecq risen...
4,business,High fuel prices hit BA's profits,british airways blamed high fuel prices 40 dro...
...,...,...,...
2220,tech,New consoles promise big problems,making games future consoles require graphic a...
2221,tech,Spam e-mails tempt net shoppers,computer users across world continue ignore se...
2222,tech,Be careful how you code,a new european directive could put software wr...
2223,tech,Losing yourself in online gaming,online role playing games time consuming enthr...


In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
df2['categories'] = enc.fit_transform(df2['categories'])

df2.head()

,categories,title,article
0,0,Ad sales boost Time Warner profit,quarterly profits us media giant timewarner ju...
1,0,Dollar gains on Greenspan speech,the dollar hit highest level euro almost three...
2,0,Yukos unit buyer faces loan claim,the owners embattled russian oil giant yukos a...
3,0,Pernod takeover talk lifts Domecq,shares uk drinks food firm allied domecq risen...
4,0,High fuel prices hit BA's profits,british airways blamed high fuel prices 40 dro...
